In [1]:
# !pip install line_profiler memory_profiler

In [1]:
%load_ext line_profiler
%load_ext memory_profiler
%load_ext autoreload
%autoreload 2

In [2]:
from pathlib import Path
import pandas as pd
import numpy as np

import plotly.graph_objects as go
from plotly.subplots import make_subplots

from plotly_resampler import FigureResampler
from plotly_resampler.downsamplers import EveryNthPoint, LTTB

/home/jonas/git/gIDLab/plotly-dynamic-resampling/.venv/lib/python3.9/site-packages/dash_bootstrap_components/_table.py:5: UserWarning: 
The dash_html_components package is deprecated. Please replace
`import dash_html_components as html` with `from dash import html`
  import dash_html_components as html


In [5]:
df_gusb = pd.read_parquet("data/df_gusb.parquet")
df_data_pc = pd.read_parquet("data/df_pc_test.parquet")

n = 110_000#_000
np_series = np.array(
    (3 + np.sin(np.arange(n) / 200_000) + np.random.randn(n) / 10) * np.arange(n) / 100_000,
    dtype=np.float32,
)
x = np.arange(len(np_series))

In [6]:
fig = FigureResampler(go.Figure(), verbose=True)

fig.add_trace(
    go.Scattergl(x=[], y=[], name="sin", showlegend=True),
    hf_x=np.arange(len(np_series)),
    hf_y=np_series,
)

fig.update_layout(height=600)
fig.show_dash(mode='external', debug=True)

	[i] DOWNSAMPLE sin	110000->1000
Dash app running on http://127.0.0.1:8050/


/home/jonas/git/gIDLab/plotly-dynamic-resampling/.venv/lib/python3.9/site-packages/jupyter_dash/jupyter_app.py:139: UserWarning:

The 'environ['werkzeug.server.shutdown']' function is deprecated and will be removed in Werkzeug 2.1.



----------------------------------------------------------------------------------------------------
 changed layout {'autosize': True}
----------------------------------------------------------------------------------------------------
 changed layout {'autosize': True}
----------------------------------------------------------------------------------------------------
 changed layout {'xaxis.range[0]': 27761.291963377415, 'xaxis.range[1]': 31375.8494404883}
----------------------------------------------------------------------------------------------------
 changed layout {'xaxis.range[0]': 27761.291963377415, 'xaxis.range[1]': 63906.86673448626}
----------------------------------------------------------------------------------------------------
 changed layout {'xaxis.range[0]': 27761.291963377415, 'xaxis.range[1]': 115854.92832701378}
----------------------------------------------------------------------------------------------------
 changed layout {'xaxis.range[0]': 74959.6457967

In [4]:
fig = FigureResampler(
    make_subplots(
        rows=2, cols=2, 
        specs=[[{}, {}], [{"colspan": 2}, None]],
        subplot_titles=("GUSB swimming pool", "Generated sine", "Power consumption"),
        vertical_spacing=0.12,
    ), default_n_shown_samples=1000, verbose=True)


# ------------ swimming pool data -----------
df_gusb_pool = df_gusb[df_gusb.zone == "zwembad"]
df_gusb_pool = df_gusb_pool[df_gusb_pool["aantal aanwezigen"] < 3_000].last('4D')
fig.add_trace(
    go.Scattergl(
        x=df_gusb_pool.index, 
        y=df_gusb_pool["aantal aanwezigen"].astype('uint16'), 
        mode='markers',
        marker_size=5,
        name="occupancy",
        showlegend=True
    ),
    hf_hovertext='mean last hour: ' + df_gusb_pool["aantal aanwezigen"].rolling('1h').mean().astype(int).astype(str),
    downsampler=EveryNthPoint(interleave_gaps=False),
    row=1,
    col=1,
)
fig.update_yaxes(title_text='Occupancy', row=1, col=1)


# ----------------- generated sine -----------
fig.add_trace(
    go.Scattergl(name="sin", line_color='#26b2e0'),
    hf_x=np.arange(len(np_series)),
    hf_y=np_series,
    row=1,
    col=2,
)

# ------------- Power consumption data -------------
df_data_pc = df_data_pc.last('190D')
for i, c in enumerate(df_data_pc.columns):
    fig.add_trace(
        go.Scattergl(name=f"room {i+1}",),
        hf_x=df_data_pc.index, hf_y=df_data_pc[c],
        row=2, col=1,
        downsampler=LTTB(interleave_gaps=True)
    )

fig.update_layout(height=600)
fig.update_yaxes(title_text='Watt/hour', row=2, col=1)
fig.update_layout(
    title='<b>Plotly-Resampler demo</b>', title_x=0.5, legend_traceorder='normal',
)

fig.show_dash(mode='external', debug=True, port=9010)

	[i] DOWNSAMPLE occupancy	10072->1000
	[i] DOWNSAMPLE sin	110000->1000
	[i] DOWNSAMPLE room 1	261133->1000
	[i] DOWNSAMPLE room 2	261133->1000
	[i] DOWNSAMPLE room 3	261133->1000
Dash app running on http://127.0.0.1:9010/
----------------------------------------------------------------------------------------------------
 changed layout {'autosize': True}
----------------------------------------------------------------------------------------------------
 changed layout {'autosize': True}
----------------------------------------------------------------------------------------------------
 changed layout {'xaxis.range[0]': '2021-11-18 12:12:27.9708', 'xaxis.range[1]': '2021-11-18 17:21:55.3749'}
----------------------------------------------------------------------------------------------------
 changed layout {'autosize': True}


In [10]:
idx_list = [0, 2]
current_graph = fig._fig_dict

cg = [current_graph['data'][idx].copy() for idx in idx_list]

In [14]:
fig._fig_dict['layout']

{'annotations': [{'font': {'size': 16},
   'showarrow': False,
   'text': 'GUSB swimming pool',
   'x': 0.225,
   'xanchor': 'center',
   'xref': 'paper',
   'y': 1,
   'yanchor': 'bottom',
   'yref': 'paper'},
  {'font': {'size': 16},
   'showarrow': False,
   'text': 'Generated sine',
   'x': 0.775,
   'xanchor': 'center',
   'xref': 'paper',
   'y': 1,
   'yanchor': 'bottom',
   'yref': 'paper'},
  {'font': {'size': 16},
   'showarrow': False,
   'text': 'Power consumption',
   'x': 0.5,
   'xanchor': 'center',
   'xref': 'paper',
   'y': 0.44,
   'yanchor': 'bottom',
   'yref': 'paper'}],
 'template': {'data': {'barpolar': [{'marker': {'line': {'color': '#E5ECF6',
       'width': 0.5},
      'pattern': {'fillmode': 'overlay', 'size': 10, 'solidity': 0.2}},
     'type': 'barpolar'}],
   'bar': [{'error_x': {'color': '#2a3f5f'},
     'error_y': {'color': '#2a3f5f'},
     'marker': {'line': {'color': '#E5ECF6', 'width': 0.5},
      'pattern': {'fillmode': 'overlay', 'size': 10, 'solid

In [9]:
fig.to_dict()['layout']

{'annotations': [{'font': {'size': 16},
   'showarrow': False,
   'text': 'GUSB swimming pool',
   'x': 0.225,
   'xanchor': 'center',
   'xref': 'paper',
   'y': 1.0,
   'yanchor': 'bottom',
   'yref': 'paper'},
  {'font': {'size': 16},
   'showarrow': False,
   'text': 'Generated sine',
   'x': 0.775,
   'xanchor': 'center',
   'xref': 'paper',
   'y': 1.0,
   'yanchor': 'bottom',
   'yref': 'paper'},
  {'font': {'size': 16},
   'showarrow': False,
   'text': 'Power consumption',
   'x': 0.5,
   'xanchor': 'center',
   'xref': 'paper',
   'y': 0.44,
   'yanchor': 'bottom',
   'yref': 'paper'}],
 'template': {'data': {'barpolar': [{'marker': {'line': {'color': '#E5ECF6',
       'width': 0.5},
      'pattern': {'fillmode': 'overlay', 'size': 10, 'solidity': 0.2}},
     'type': 'barpolar'}],
   'bar': [{'error_x': {'color': '#2a3f5f'},
     'error_y': {'color': '#2a3f5f'},
     'marker': {'line': {'color': '#E5ECF6', 'width': 0.5},
      'pattern': {'fillmode': 'overlay', 'size': 10, 's

In [4]:
np_series_r = np_series.astype(int)

In [7]:
def test_series(i):
    idx = pd.Index(x, name='timestamp', copy=False)
    s = pd.Series(np_series, idx, copy=False, name='data')

In [9]:
%lprun -f test_series test_series(0)

UsageError: Could not find function 'test_series'.
NameError: name 'test_series' is not defined


In [6]:
%mprun -cf fig.add_trace fig.add_trace(go.Scattergl(name="sin", line_color='#26b2e0'), hf_x=x,hf_y=np_series_r)

Filename: /home/jonas/git/gIDLab/plotly-dynamic-resampling/plotly_resampler/figure_resampler.py

Line #    Mem usage    Increment  Occurences   Line Contents
   300   2241.7 MiB   2241.7 MiB           1       def add_trace(
   301                                                 self,
   302                                                 trace,
   303                                                 max_n_samples: int = None,
   304                                                 downsampler: AbstractSeriesDownsampler = None,
   305                                                 limit_to_view: bool = False,
   306                                                 # Use these if you want some speedups (and are working with really large data)
   307                                                 hf_x: Iterable = None,
   308                                                 hf_y: Iterable = None,
   309                                                 hf_hovertext: Union[str, Iterable] = Non

In [8]:
%lprun -f fig.add_trace fig.add_trace(go.Scattergl(name="sin", line_color='#26b2e0'), hf_x=x,hf_y=np_series_r)

Timer unit: 1e-06 s

Total time: 0.169296 s
File: /home/jonas/git/gIDLab/plotly-dynamic-resampling/plotly_resampler/figure_resampler.py
Function: add_trace at line 300

Line #      Hits         Time  Per Hit   % Time  Line Contents
   300                                               def add_trace(
   301                                                   self,
   302                                                   trace,
   303                                                   max_n_samples: int = None,
   304                                                   downsampler: AbstractSeriesDownsampler = None,
   305                                                   limit_to_view: bool = False,
   306                                                   # Use these if you want some speedups (and are working with really large data)
   307                                                   hf_x: Iterable = None,
   308                                                   hf_y: Iterable = None,
   

In [43]:
fig = FigureResampler(go.Figure())

In [44]:
for i in range(10):
    print(i)
    fig.add_trace(go.Scattergl(name="sin", line_color='#26b2e0'), hf_x=x,hf_y=np_series_r)

0
1
2
3
4
5
6
7
8
9


In [45]:
fig.show_dash(mode='external')

/home/jonas/git/gIDLab/plotly-dynamic-resampling/.venv/lib/python3.9/site-packages/jupyter_dash/jupyter_app.py:139: UserWarning:

The 'environ['werkzeug.server.shutdown']' function is deprecated and will be removed in Werkzeug 2.1.



Dash app running on http://127.0.0.1:8050/


In [23]:
from plotly.io._utils import validate_coerce_fig_to_dict

In [32]:
from plotly.io._renderers import renderers

In [35]:
%%timeit -n 10 -r 10

3.27 ms ± 691 µs per loop (mean ± std. dev. of 10 runs, 10 loops each)


In [42]:
fig_dict['application/vnd.plotly.v1+json']['layout'].keys()

dict_keys(['template'])

In [36]:
fig_dict = renderers._build_mime_bundle(fig.to_dict())

In [28]:
validate_coerce_fig_to_dict(fig, True)['layout']

{'template': {'data': {'barpolar': [{'marker': {'line': {'color': '#E5ECF6',
       'width': 0.5},
      'pattern': {'fillmode': 'overlay', 'size': 10, 'solidity': 0.2}},
     'type': 'barpolar'}],
   'bar': [{'error_x': {'color': '#2a3f5f'},
     'error_y': {'color': '#2a3f5f'},
     'marker': {'line': {'color': '#E5ECF6', 'width': 0.5},
      'pattern': {'fillmode': 'overlay', 'size': 10, 'solidity': 0.2}},
     'type': 'bar'}],
   'carpet': [{'aaxis': {'endlinecolor': '#2a3f5f',
      'gridcolor': 'white',
      'linecolor': 'white',
      'minorgridcolor': 'white',
      'startlinecolor': '#2a3f5f'},
     'baxis': {'endlinecolor': '#2a3f5f',
      'gridcolor': 'white',
      'linecolor': 'white',
      'minorgridcolor': 'white',
      'startlinecolor': '#2a3f5f'},
     'type': 'carpet'}],
   'choropleth': [{'colorbar': {'outlinewidth': 0, 'ticks': ''},
     'type': 'choropleth'}],
   'contourcarpet': [{'colorbar': {'outlinewidth': 0, 'ticks': ''},
     'type': 'contourcarpet'}],
  

In [5]:
%%timeit -n 3 -r 10
fig.to_dict()

NameError: name 'fig' is not defined

In [19]:
fig.to_dict()['layout']

{'template': {'data': {'barpolar': [{'marker': {'line': {'color': '#E5ECF6',
       'width': 0.5},
      'pattern': {'fillmode': 'overlay', 'size': 10, 'solidity': 0.2}},
     'type': 'barpolar'}],
   'bar': [{'error_x': {'color': '#2a3f5f'},
     'error_y': {'color': '#2a3f5f'},
     'marker': {'line': {'color': '#E5ECF6', 'width': 0.5},
      'pattern': {'fillmode': 'overlay', 'size': 10, 'solidity': 0.2}},
     'type': 'bar'}],
   'carpet': [{'aaxis': {'endlinecolor': '#2a3f5f',
      'gridcolor': 'white',
      'linecolor': 'white',
      'minorgridcolor': 'white',
      'startlinecolor': '#2a3f5f'},
     'baxis': {'endlinecolor': '#2a3f5f',
      'gridcolor': 'white',
      'linecolor': 'white',
      'minorgridcolor': 'white',
      'startlinecolor': '#2a3f5f'},
     'type': 'carpet'}],
   'choropleth': [{'colorbar': {'outlinewidth': 0, 'ticks': ''},
     'type': 'choropleth'}],
   'contourcarpet': [{'colorbar': {'outlinewidth': 0, 'ticks': ''},
     'type': 'contourcarpet'}],
  

In [15]:
%%timeit -n 3 -r 10
fig.to_plotly_json()

679 µs ± 36.8 µs per loop (mean ± std. dev. of 10 runs, 3 loops each)


In [42]:
# fig = FigureResampler(
#     make_subplots(
#         rows=3, cols=2, 
#         specs=[[{}, {}], [{"colspan": 2}, None], [{"colspan": 2}, None]],
#         subplot_titles=("GUSB swimming pool", "Generated sine", "Energy consumption sensors", "Energy consumption prediction"),
#         vertical_spacing=0.12,
#     ), default_n_shown_samples=1000, verbose=False)


# # ------------ swimming pool data -----------
# df_gusb_pool = df_gusb[df_gusb.zone == "zwembad"]
# df_gusb_pool = df_gusb_pool[df_gusb_pool["aantal aanwezigen"] < 3_000].first('3D')
# fig.add_trace(
#     go.Scattergl(
#         x=df_gusb_pool.index, 
#         y=df_gusb_pool["aantal aanwezigen"].astype('uint16'), 
#         mode='markers',
#         marker_size=5,
#         name="occupancy",
#         showlegend=True
#     ),
#     hf_hovertext='mean last hour: ' + df_gusb_pool["aantal aanwezigen"].rolling('1h').mean().astype(int).astype(str),
#     downsampler=EveryNthPoint(interleave_gaps=False),
#     row=1,
#     col=1,
# )
# fig.update_yaxes(title_text='Occupancy', row=1, col=1)


# # ----------------- generated sine -----------
# fig.add_trace(
#     go.Scattergl(name="sin", line_color='#26b2e0'),
#     hf_x=np.arange(len(np_series)),
#     hf_y=np_series,
#     row=1,
#     col=2,
# )

# # ------------- Power consumption data -------------
# df_data_pc = df_data_pc.first('190D')
# for i, c in enumerate(df_data_pc.columns):
#     fig.add_trace(
#         go.Scattergl(name=f"room {i+1}",),
#         hf_x=df_data_pc.index, hf_y=df_data_pc[c],
#         row=2, col=1,
#         downsampler=LTTB(interleave_gaps=True)
#     )

# # ------------- Power consumption predictoins -------------
# df_pc = df_pc.first('190D')
# fig.add_trace(
#     go.Scattergl(name="target", line_color='#85ab8a'),
#     hf_x=df_pc.index, hf_y=df_pc['avg_15min_GAP'],
#     row=3, col=1
# )

# fig.add_trace(
#     go.Scattergl(
#         x=df_pc.index, y=df_pc.predictions,
#         name="predictions", marker_color="salmon",
#         legendgroup='predictions'
#     ),
#     row=3, col=1
# )

# fig.add_trace(
#     go.Scatter(
#         x=df_pc.index,
#         y=df_pc.predictions + df_pc["std"],
#         name="std-bound",
#         showlegend=False,
#         marker_color='black',
#         line=dict(width=0),
#         mode='lines',
#         legendgroup='predictions'
#     ),
#     row=3, col=1
# )

# fig.add_trace(
#     go.Scatter(
#         x=df_pc.index,
#         y=df_pc.predictions - df_pc["std"],
#         name="std-bound",
#         showlegend=False,
#         marker_color='black',
#         line=dict(width=0),
#         mode='lines',
#         fillcolor='rgba(162, 161, 171, 0.35)',
#         opacity=0.2,
#         fill='tonexty',
#         legendgroup='predictions'
#     ),
#     row=3, col=1
# )

# fig.update_layout(height=600)
# fig.update_yaxes(title_text='Wh', row=2, col=1)
# fig.update_yaxes(title_text='kWh', row=3, col=1)

# fig.update_xaxes(matches='x4', row=2, col=1, showticklabels=False)
# fig.update_yaxes(row=1, col=2, showticklabels=False)
# fig.update_layout(
#     title='<b>Plotly-Resampler demo</b>', title_x=0.5, legend_traceorder='normal',
# )

# fig.show_dash(mode='inline')

In [ ]:
fig = FigureResampler(go.Figure(), verbose=True)

fig.add_trace(
    go.Scattergl(x=[], y=[], name="sin", showlegend=True),
    hf_x=np.arange(len(np_series)),
    hf_y=np_series,
)

fig.update_layout(height=600)
fig.show_dash(mode='inline')
